# Neural Network Test

- Imports 

In [1]:
import numpy as np 
import pandas as pd

Load Data

In [2]:
train_df= pd.read_csv(r"C:/project/datasets/train_nn.csv")
test_df= pd.read_csv(r"C:/project/datasets/test_nn.csv")

X_train1 = train_df.drop(["target_cls","target_reg"],axis=1).values.astype(float)
X_train= X_train1[:,1:]
y_cls= train_df["target_cls"].values.astype(int)
y_reg= train_df["target_reg"].values.astype(float).reshape(-1,1)
X_test1 = test_df.values.astype(float)
X_test = X_test1[:,1:]


print(X_train.shape , y_cls.shape,y_reg.shape, X_test.shape)
print()
print(X_train[:5])

(16000, 20) (16000,) (16000, 1) (4000, 20)

[[ 4.    7.    5.    5.    4.    6.    7.    3.    7.   11.    8.    9.
   3.    8.    4.    8.   15.7   8.75  4.69 32.7 ]
 [ 4.    7.    6.    5.    5.    6.    8.    3.    7.   11.    8.    9.
   3.    8.    4.    7.   18.84  8.75  4.69 32.7 ]
 [ 3.    5.    4.    3.    3.    7.    8.    5.    5.    7.    7.    6.
   5.    9.    2.    6.   12.56  6.25  3.35 52.5 ]
 [ 4.   10.    6.    7.    6.    5.    7.    5.    7.    6.    6.   12.
   3.    8.    6.    9.   18.84  8.75  6.7  54.  ]
 [ 4.   10.    6.    8.    4.    8.   11.    2.    3.    4.   10.    9.
   3.   11.    1.    8.   18.84  3.75  6.7  23.4 ]]


Pipeline preprocessing

In [3]:
from  ml_lib.preprocessing.pipeline import Pipeline
from ml_lib.preprocessing.imputer import SimpleImputer
from ml_lib.preprocessing.scaler import StandardScaler

pipe = Pipeline([
   ( 'imputer',SimpleImputer(strategy="mean")),
   ('scaler',StandardScaler())

])
X_train_tr= pipe.fit(X_train)



ONe Hot Encoding

In [18]:
from ml_lib.preprocessing.encoding import OneHotEncoder,LabelEncoder

classes = np.unique(y_cls)
le= LabelEncoder()
y_cls_enc=le.fit_transform(y_cls)


encd= OneHotEncoder()
y_cls_oh=encd.fit_transform(y_cls_enc.reshape(-1,1))

classes=le.classes_
num_classes= len(classes)

print("Integer encoded labels:", np.unique(y_cls_enc))
print("Classes mapping:", classes)
print("One-hot shape:", y_cls_oh.shape)
print("No of classes",num_classes)



Integer encoded labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25]
Classes mapping: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26]
One-hot shape: (16000, 26)
No of classes 26


### MOdel GEnration

In [5]:
from ml_lib.neural_network import Sequential,Dense,MSE,CategoricalCrossEntropy,BinaryCrossEntropy,Model

shared=Sequential([
    Dense(128,activation='relu',init='he'),
    Dense(64,activation='relu',init='he'),
])

classification_head= Sequential([
    Dense(32,activation='relu',init='he'),
    Dense(num_classes,activation='softmax',init='xavier')

])

regression_head=Sequential([
    Dense(32,activation='relu',init='he'),
    Dense(1,init='xavier') # linear output for regression
])

model= Model(
    shared_layers = shared,
    classification_head=classification_head,
    regression_head=regression_head
)

- Compile Model

In [6]:
model.compile (
    optimizer=('adam'),losses=[CategoricalCrossEntropy(),MSE()],loss_weights=[0.6,0.4],lr=0.005
)

- Train the model

In [8]:
model.fit(
    X_train_tr,[y_cls_oh,y_reg],
    epochs=700,
    verbose=10
)

Epoch 1/700 | Cls Loss: 2.4845 | Reg Loss: 1.3222
Epoch 11/700 | Cls Loss: 2.4234 | Reg Loss: 1.2117
Epoch 21/700 | Cls Loss: 2.3606 | Reg Loss: 1.1204
Epoch 31/700 | Cls Loss: 2.2930 | Reg Loss: 1.0444
Epoch 41/700 | Cls Loss: 2.2315 | Reg Loss: 0.9676
Epoch 51/700 | Cls Loss: 2.1765 | Reg Loss: 0.8949
Epoch 61/700 | Cls Loss: 2.1277 | Reg Loss: 0.8338
Epoch 71/700 | Cls Loss: 2.0829 | Reg Loss: 0.7866
Epoch 81/700 | Cls Loss: 2.0388 | Reg Loss: 0.7491
Epoch 91/700 | Cls Loss: 1.9910 | Reg Loss: 0.7226
Epoch 101/700 | Cls Loss: 1.9414 | Reg Loss: 0.7007
Epoch 111/700 | Cls Loss: 1.8953 | Reg Loss: 0.6799
Epoch 121/700 | Cls Loss: 1.8485 | Reg Loss: 0.6579
Epoch 131/700 | Cls Loss: 1.8027 | Reg Loss: 0.6388
Epoch 141/700 | Cls Loss: 1.7549 | Reg Loss: 0.6235
Epoch 151/700 | Cls Loss: 1.7090 | Reg Loss: 0.6119
Epoch 161/700 | Cls Loss: 1.6656 | Reg Loss: 0.6072
Epoch 171/700 | Cls Loss: 1.6219 | Reg Loss: 0.5950
Epoch 181/700 | Cls Loss: 1.5758 | Reg Loss: 0.5830
Epoch 191/700 | Cls Los

Check the accuracy


In [10]:
cls_pred,reg_pred = model.predict(X_train_tr)

y_pred = np.argmax(cls_pred, axis=1)
y_pred_labels=le.inverse_transform(y_pred)
from ml_lib.metrics.math import r2_score
from ml_lib.metrics.validation import f1_score_multiclass
print(y_cls,'\n',y_pred_labels)

classification_f1_score = f1_score_multiclass (y_cls,y_pred_labels)
regression_R2= r2_score(y_reg,reg_pred)

print("classification F1 Score = ",classification_f1_score, 'regression R2: ',regression_R2 )

[ 5  5 14 ... 24  9 10] 
 [ 5  5 14 ... 24  9 10]
classification F1 Score =  0.8522150574484834 regression R2:  0.9260314194891509


## Predict Data

In [13]:
print(X_test.shape)
X_test_tr=pipe.transform(X_test)
test_pred_cls,test_pred_reg=model.predict(X_test_tr)

y_pred_t=np.argmax(test_pred_cls,axis=1)
y_cls_t=le.inverse_transform(y_pred_t)

print(y_cls_t.shape,test_pred_cls.shape)



(4000, 20)
(4000,) (4000, 26)


- Save the data as csv file

In [16]:
test_df_clean=test_df.iloc[:,1:]

test_df_clean['target_cls']=y_cls_t
test_df_clean['target_reg']=test_pred_reg

test_df_clean.to_csv(r'C:/project/datasets/processed/nn_predictons.csv')